<a href="https://colab.research.google.com/github/kimdonggyu2008/generative-ai-python/blob/main/Greedy_encoder_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchdata torchtext portalocker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 37.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

def generate_tokens(text_iter,language):
  language_index={SRC_LANGUAGE:0, TGT_LANGUAGE:1}

  for text in text_iter:
    yield token_transformer[language](text[language_index[language]])

SRC_LANGUAGE="de"
TGT_LANGUAGE="en"
UNK_IDX,PAD_IDX,BOS_IDX,EOS_IDX=0,1,2,3
special_symbols=["<unk>","<pad>","<bos>","<eos>"]

token_transform={
    SRC_LANGUAGE:get_tokenizer("spacy",language="de_core_news_sm"),
    TGT_LANGUAGE:get_tokenizer("spacy",language="en_core_web_sm"),
}
print("Token Transform:")
print(token_transform)


vocab_transform={}
for language in [SRC_LANGUAGE,TGT_LANGUAGE]:
  train_iter=Multi30k(split="train",language_pair=(SRC_LANGUAGE,TGT_LANGUAGE))
  vocab_transform[language]=build_vocab_from_iterator(
      generate_tokens(train_iter,language),
      min_freq=1,
      specials=special_symbols,
      special_first=True,
  )

for language in [SRC_LANGUAGE,TGT_LANGUAGE]:
  vocab_transform[language].set_default_index(UNK_IDX)

print("vocab transform")
print(vocab_transform)

OSError: [E050] Can't find model 'de_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
import math
import torch
from torch import nn

class PositionalEncoding(nn.Module):
  def __init__(self,d_model,max_len,dropout=0.1):
    super().__init__()
    self.dropout=nn.Module(p=dropout)

    positional=torch.arange(max_len).unsqueeze(1)
    div_term=torch.exp(
        torch.aragne(0,d_model,2)*(-math.log(10000.0)/d_model)
    )

    pe.torch.zeros(max_len,1,d_model)
    pe[:,0,0::2]=torch.sin(position*div_term)
    pe[:,0,1::2]=torch.cos(position*div_term)
    self.register_buffer("pe",pe)

  def forward(self,x):
    x=x+self.pe[:x.size(0)]
    return self.dropout(x)

class TokenEmbedding(nn.Module):
  def __init__(self,vocab_size,emb_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,emb_size)
    self.emb_size=emb_size

  def forward(self,tokens):
    return self.embedding(tokens.long())*math.sqrt(self.emb_size)


class Seq2SeqTransformater(nn.Module):
  def __init__(
      self,
      num_encoder_layers,
      num_decoder_layers,
      emb_size,
      max_len,
      nhead,
      src_vocab_size,
      tgt_vocab_size,
      dim_feedforward,
      dropout=0.1
  ):
    super().__init__()
    self.src_tok_emb=TokenEmbedding(src_vocab_size,emb_size)
    self.tgt_tok_emb=TokenEmbedding(tgt_vocab_size,emb_size)
    self.positional_encoding=PositionalEncoding(
        d_model=emb_size,max_len=max_len,dropout=dropout
    )

    self.transformer=nn.Transformer(
        d_model=emb_size,
        nhead=nhead,
        num_encoder_layers=num_encoder_layers,
        num_decoder_layers=num_decoder_layers,
        dim_feedforward=dim_feedforward,
        dropout=dropout
    )
    self.generator=nn.Linear(emb_size,tgt_vocab_size)

  def forward(
    self,
    src,
    trg,
    src_mask,
    tgt_mask,
    src_padding_mask,
    tgt_padding_mask,
    memory_key_padding_mask,
  ):
    src_emb=self.positional_encoding(self.src_tok_emb(src))
    tgt_emb=self.positional_encoding(self.tgt_tok_emb(trg))
    outs=self.transformer(
        src=src_emb,
        tgt=tgt_emb,
        src_mask=src_mask,
        memory_mask=None,
        src_key_padding_mask=src_padding_mask,
        tgt_key_padding_mask=memory_key_padding_mask

    )
    return self.generator(outs)

  def encode(self,src,src_mask):
    return self.transformer.encoder(
        self.positional_encoding(self.src_tok_emb(src)),src_mask
    )

  def decode(self,tgt,memory,tgt_mask):
    return self.transformer.decoder(
        self.positional_encoding(self.tgt_tok_emb(tgt)),memory,tgt_mask
    )

In [ ]:
transformer=torch.nn.Transformer(
    d_model=512,
    nhead=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    dim_feedforward=2048,
    dropout=0.1,
    activation=torch.nn.functional.relu,
    layer_norm_eps=1e-05,
)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def sequential_transformers(*transformers):
  def func(txt_input):
    for tranform in transforms:
      txt_input=transform(txt_input)
    return txt_input
  return func

def input_transform(token_ids):
  return torch.cat(
      (torch.tensor([BOS_IDX]),torch.tensor(token_ids),torch.tensor([EOS_IDX]))
  )

def collator(batch):
  src_batch,tgt_batch=[],[]
  for src_sample,tgt_sample in batch:
    src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
    tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))
  src_batch=pad_sequence(src_batch,padding_value=PAD_IDX)
  tgt_batch=pad_sequence(tgt_batch,padding_value=PAD_IDX)
  return src_batch,tgt_batch

text_transform={}
for language in [SRC_LANGUAGE,TGT_LANGUAGE]:
  text_transform[language]=sequential_transforms(
      token_transform[language],vocab_transform[language],input_transform
  )

data_iter=Multi30k(split="valid",language_pair(SRC_LANGUAGE,TGT_LANGUAGE))
dataloader=DataLoader(data_iter,batch_size=BATCH_SIZE,collate_fn=collator)
source_tensor,target_tensor=next(iter(dataloader))

print("(source,target):")
print(next(iter(data_iter)))

print("soruce_batch:", source_tensor.shape)
print(source_tensor)

print("target_batch:",target_tensor.shape)
print(target_tensor)


SyntaxError: positional argument follows keyword argument (<ipython-input-10-2b9f76a9a59a>, line 31)

In [ ]:
def generate_square_subsequent_mask(s):
  mask=(torch.triu(torch.ones((s,s),device=DEVICE))==1).transpose(0,1)
  mask=(
      mask.float()
      .masked_fill(mask==0,float("-inf"))
      .masked_fill(mask==1,float(0,0))
  )
  return mask


def create_mask(src,tgt):
  src_seq_len=src.shape[0]
  tgt_seq_len=tgt.shape[0]

  tgt_mask=generate_square_subsequent_mask(tgt_seq_len)
  src_mask=torch.zeros((src_seq_len,src_seq_len),device=DEVICE).type(torch.bool)

  src_padding_mask=(src==PAD_IDX).transpose(0,1)
  tgt_padding_mask=(tgt==PAD_IDX).transpose(0,1)
  return src_mask,tgt_mask,src_padding_mask,tgt_padding_mask

target_input=target_tensor[:-1,:]
target_out=target_tensor[1:,:]

source_mask,target_mask,source_padding_mask,target_padding_mask=create_mask(
    source_tensor,target_input
)

print("source_mask:",source_mask.shape)
print(source_mask)

print("target_mask:",target_mask.shape)
print(target_mask)

print("source_padding_mask:",source_padding_mask.shape)
print(source_padding_mask)


print("target_padding_mask:",target_padding_mask.shape)
print(target_padding_mask)

NameError: name 'target_tensor' is not defined

In [ ]:
def run(model,optimizer,driterion,split):
  model.train() if split=="train"else model.eval()
  data_iter=Multi30k(split=split,language_pair=(SRC_LANGUAGE,TGT_LANGUAGE))
  dataloader=DataLoader(data_iter,batch_size=BATCH_SIZE,collate_fn=collator)

  losses=0
  for source_batch,target_batch in dataloader:
    source_batch=source_batch.to(DEVICE)
    target_batch=target_batch.to(DEVICE)

    target_input=target_batch[:-1,:]
    target_output=target_batch[1:,:]

    src_mask,tgt_mask,src_padding_mask,tgt_padding_mask=create_mask(
        source_batch,target_input
    )

    logits=model(
        src=source_batch,
        trg=target_input,
        src_mask=src_mask,
        tgt_mask=tgt_mask,
        src_padding_mask=src_padding_mask,
        tgt_padding_mask=tgt_padding_mask,
        memory_key_padding_mask=src_padding_mask,
    )
    optimizer.zero_grad()
    loss=criterion(logits.reshape(-1,logits.shape[-1]),target_output.reshape(-1))
    if split=="train":
      loss.backward()
      optimizer.step()
    losses+=loss.item()
  return losses/len(list(dataloader))

for epoch in range(5):
  train_loss=run(model,optimizer,criterion,"train")
  val_loss=run(model,optimizer,criterion,"valid")
  print(f"Epoch:{epoch+1}, Train loss:{train_loss:.3f}, Val loss:{val_loss:.3f}")

NameError: name 'model' is not defined

그리드 디코딩

각 단어별로 가장 높은 확률을 가지는 단어를 선택하는 방식으로 번역 진행



In [ ]:
def greedy_decode(model,source_tensor,source_mask,max_len,start_symbol):
  source_tensor=source_tensor.to(DEVICE)
  source_mask=source_mask.to(DEVICE)

  memory=model.encode(source_tensor,source_mask)
  ys=torch.ones(1,1).fill_(start_symbol).type(torch.long).to(DEVICE)
  for i in range(max_len-1):
    memory=memory.to(DEVICE)
    target_mask=generate_square_subsequent_mask(ys.size(0))
    target_mask=target_mask.type(torch.bool).to(DEVICE)

    out=model.decode(ys,memory,target_mask)
    out=out.transpose(0,1)
    prob=model.generator(out[:,-1])
    _,next_word=torch.max(prob,dim=1)
    next_word=next_word.item()

    ys=torch.cat(
        [ys,torch.ones(1,1).type_as(source_tensor.data).fill_(next_word)],dim=0
    )
    if next_word==EOS_IDX:
      break
  return ys


def translate(model,source_sentence):
  model.eval()
  source_tensor=text_transform[SRC_LANGUAGE](source_sentence).view(-1,1)
  num_tokens=source_tensor.shape[0]
  src_mask=(torch.zeros(num_tokens,num_tokens)).type(torch.bool)
  tgt_tokens=greedy_decode(
      model,source_tensor,src_mask,max_len=num_tokens+5,start_symbol=BOS_IDX
  ).flatten()
  output=vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))[1:-1]
  return " ".join(output)

output_oov=translate(model,"Eine Gruppe von Menschen steht vor einem Iglu.")
output=translate(model,"Eine Gruppe vone Menschen steht vor einem Gebaude.")
print(output_oov)
print(output)

NameError: name 'model' is not defined